In [1]:
import os
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/Colab Notebooks/src')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import torch
import pandas as pd
import plotly.express as px
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from sklearn.metrics import accuracy_score
from utils import load_json

In [3]:
state = load_json('model/all_state/trainer_state.json')
df = pd.DataFrame(state['log_history']).groupby('epoch').last()
df = df[['loss', 'eval_loss', 'eval_accuracy']].reset_index().rename(columns={'epoch': 'Epoch'})
df = df[['Epoch', 'loss', 'eval_loss', 'eval_accuracy']].rename(columns={'loss': 'Training Loss', 'eval_loss': 'Validation Loss', 'eval_accuracy': 'Accuracy'})
df


,Epoch,Training Loss,Validation Loss,Accuracy
0,1.0,0.4474,0.327585,0.866152
1,2.0,0.2955,0.288485,0.889148
2,3.0,0.2430,0.289060,0.887531
3,4.0,0.2077,0.277957,0.890586
4,5.0,0.1790,0.289795,0.899389
5,6.0,0.1621,0.300069,0.897952
6,7.0,0.1465,0.308901,0.898670


In [4]:
fig = px.line(df, x='Epoch', y=['Training Loss', 'Validation Loss'], title='Training Loss and Validation Loss')
fig.add_bar(x=df['Epoch'], y=df['Accuracy'], name='Accuracy', marker_color='#54A24B', width=0.5)
fig.update_layout(width=800, height=400)
fig.show()

In [5]:

def preprocess_and_predict(row, model, tokenizer, device):
  acronym = row['acronym_token']
  expansion = row['expansion']
  sentence = row['sentence']
  text = tokenizer.sep_token.join([acronym, expansion, sentence])
  inputs = tokenizer.encode_plus(text, truncation=True, return_tensors='pt').to(device)
  with torch.no_grad():
    outputs = model(**inputs)
  logits = outputs.logits
  prediction = torch.argmax(logits, dim=1).item()
  return prediction

def evaluate(df, model, tokenizer, device):
  predictions = df.apply(lambda row: preprocess_and_predict(row, model, tokenizer, device), axis=1)
  score = accuracy_score(df['label'], predictions)
  return score

In [6]:
# load data
training_set = pd.read_parquet('../dataset/parquet/train.parquet')
testing_set = pd.read_parquet('../dataset/parquet/test.parquet')
training_set['label'] = training_set['label'].astype(int)
testing_set['label'] = testing_set['label'].astype(int)

In [7]:
model_name = 'model/checkpoint-2840'
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(model_name).to(device)
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [11]:
train_score = evaluate(training_set, model, tokenizer, device)
test_score = evaluate(testing_set, model, tokenizer, device)

In [12]:
print(f"Accuracy - train: {train_score}, test: {test_score}")

Accuracy - train: 0.9466109154929577, test: 0.8905856988860942
